<a href="https://colab.research.google.com/github/Arun-purakkatt/Applied-AI-Course/blob/master/Amazon_fine_food_reviews_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


In [0]:
#Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# using the SQLite Table to read data.
# using the SQLite Table to read data.
con = sqlite3.connect('/content/drive/My Drive/database.sqlite') 
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
# SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000, will give top 500000 data points
# you can change the number to any other number based on your computing power

# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000""", con) 
# for tsne assignment you can take 5k data points


In [0]:
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 

In [0]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
  if x<3:   
    return 'negative'
  return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore=filtered_data['Score']
positiveNegative=actualScore.map(partition)
filtered_data['Score']=positiveNegative

In [7]:
print(filtered_data.shape)
filtered_data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Exploratory Data Analysis

Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data. Following is an example:

In [8]:
display=pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score !=3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that

ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)

ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [0]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')

In [10]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

In [11]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100


69.25890143662969

Observation:- It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [12]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)
display


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [0]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]


In [14]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()


(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

7.2.3 Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

Begin by removing the html tags

Remove any punctuations or limited set of special characters like , or . or # etc.

Check if the word is made up of english letters and is not alpha-numeric

Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)

Convert the word to lowercase

Remove Stopwords

Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)

After which we collect the words used to describe positive and negative reviews



In [15]:
# find sentences containing HTML tags
i=0
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
       print(i)
       print(sent)
       break;
    i+=1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [16]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')


stop= set(stopwords.words('english'))#set of all stop words
sno = nltk.stem.SnowballStemmer('english')#Initialise snowballstemmer

def cleanhtml(sentence):#function to clean the word of any html tags
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'weren', 'out', 'this', 'of', 'so', 'doesn', 'ma', 'too', 'are', 'while', 'when', 'm', 'above', 'all', 'itself', 'myself', "that'll", 'now', 'some', 'with', 'those', 'just', "aren't", 'hadn', 'mustn', 'did', 'a', 'until', "hasn't", 'having', 'because', 'whom', 'herself', 'off', 'there', 'other', 'again', 'through', 'the', 'wouldn', "mightn't", 'had', 'am', 'from', 'wasn', 'such', 'do', 'it', 'which', 'he', 'doing', 'an', 'as', "she's", 'more', 'who', "haven't", 'no', "needn't", "couldn't", 'on', "wasn't", 'over', 'during', "hadn't", 'by', 'only', 'but', 'be', 'how', 'me', 'you', 'own', 'hasn', "weren't", 'our', 'their', 'very', "you'll", "didn't", 'were', 'd', 'each', 'my', 'o', 'if', 'your', 'its', "doesn't", 'haven', "shouldn't", 'down', 'ours', 'her', 'his', 'they', 've', 'why', 'yourself', 'most', 'didn', 'she', 'is', 'between', "shan't", 'up', "wouldn't", 'and', "don't",

In [0]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1


In [0]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review


In [0]:
final.head(3) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [0]:
# creating pickle file for this 50k data points as 
# this took huge time to compute


import pickle

pickle_out = open("cleanedData.pickle","wb")
pickle.dump(final,pickle_out)
pickle.dump(all_positive_words,pickle_out)
pickle.dump(all_negative_words,pickle_out)
pickle_out.close()

In [21]:
pickle_in=open("cleanedData.pickle","rb")
pik = pickle.load(pickle_in)
pik.head(2)
all_positive_words = pickle.load(pickle_in)
all_negative_words = pickle.load(pickle_in)
all_positive_words


[b'witti',
 b'littl',
 b'book',
 b'make',
 b'son',
 b'laugh',
 b'loud',
 b'recit',
 b'car',
 b'drive',
 b'along',
 b'alway',
 b'sing',
 b'refrain',
 b'hes',
 b'learn',
 b'whale',
 b'india',
 b'droop',
 b'love',
 b'new',
 b'word',
 b'book',
 b'introduc',
 b'silli',
 b'classic',
 b'book',
 b'will',
 b'bet',
 b'son',
 b'still',
 b'abl',
 b'recit',
 b'memori',
 b'colleg',
 b'grew',
 b'read',
 b'sendak',
 b'book',
 b'watch',
 b'realli',
 b'rosi',
 b'movi',
 b'incorpor',
 b'love',
 b'son',
 b'love',
 b'howev',
 b'miss',
 b'hard',
 b'cover',
 b'version',
 b'paperback',
 b'seem',
 b'kind',
 b'flimsi',
 b'take',
 b'two',
 b'hand',
 b'keep',
 b'page',
 b'open',
 b'fun',
 b'way',
 b'children',
 b'learn',
 b'month',
 b'year',
 b'learn',
 b'poem',
 b'throughout',
 b'school',
 b'year',
 b'like',
 b'handmot',
 b'invent',
 b'poem',
 b'great',
 b'littl',
 b'book',
 b'read',
 b'nice',
 b'rhythm',
 b'well',
 b'good',
 b'repetit',
 b'littl',
 b'one',
 b'like',
 b'line',
 b'chicken',
 b'soup',
 b'rice',
 b

# BOW : Bag Of Words

In [0]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)


In [23]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [25]:
final_counts.get_shape()


(364171, 115281)

Observation: 364171 Reviews and 115281 unique words in the reviews



# Bi-Grams and N-grams:
Now We have all positive and negative reviews in two lists, now lets get the frequency of most common positive reviews and most common negative reviews



In [0]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)

In [27]:
print("Most Common Positive Words : ",freq_dist_positive.most_common(25))
print("Most Common Negative Words : ",freq_dist_negative.most_common(25))


Most Common Positive Words :  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791), (b'tea', 83888), (b'coffe', 78814), (b'make', 75107), (b'get', 72125), (b'food', 64802), (b'would', 55568), (b'time', 55264), (b'buy', 54198), (b'realli', 52715), (b'eat', 52004), (b'amazon', 49681), (b'price', 48832), (b'also', 48025), (b'find', 48007), (b'much', 47733)]
Most Common Negative Words :  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716), (b'get', 13786), (b'buy', 13752), (b'order', 12871), (b'food', 12754), (b'dont', 11877), (b'tea', 11665), (b'even', 11085), (b'box', 10844), (b'amazon', 10073), (b'make', 9840), (b'time', 9799), (b'bag', 9739), (b'eat', 9515), (b'much', 9440), (b'realli', 9395)]


Observation:- From the above it can be seen that the most common positive and the negative words overlap for eg. 'like' could be used as 'not like' etc.
So, it is a good idea to consider pairs of consequent words (bi-grams) or q sequnce of n consecutive words (n-grams)

In [0]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# (1,2) indicates unigrams and bigrams if its (1,3) -> uni,bi and trigrams

count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [29]:
final_bigram_counts.get_shape()


(364171, 2910192)

In [0]:
#tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,3) ) #in scikit-learn
final_trigram_counts = count_vect.fit_transform(final['Text'].values)

In [0]:
count_vect = CountVectorizer(ngram_range=(1,2))
final_uni_bigrams_count = count_vect.fit_transform(final['Text'].values)


In [0]:
pickle_out = open("n_grams.pickle","wb")
pickle.dump(final_uni_bigrams_count,pickle_out)
pickle_out.close()

In [0]:
pickle_in=open("n_grams.pickle","rb")
final_uni_bigrams_count = pickle.load(pickle_in)

In [35]:
final_uni_bigrams_count.get_shape()

(364171, 2910192)

Observation: here we have 29lakhs unique uni and bigrams In the case of bag of words we got 115281 unique words



## TFIDF

In [0]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)


In [37]:
final_tf_idf.get_shape()


(364171, 2910192)

In [0]:
final_tf_idf[1]


<1x2910192 sparse matrix of type '<class 'numpy.float64'>'
	with 84 stored elements in Compressed Sparse Row format>

Observation: As it is a compressed sparse matrix we cant access the data with index.



In [38]:
# we can access the data like this
# getting all the unique words from the tf_idf_vect

features = tf_idf_vect.get_feature_names()

len(features)


2910192

In [39]:
features[100000:100010]
tf_idf_vect


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Above are the 10 words uni and bi grams



In [40]:
# To convert a row of sparse matrix to numpy array

print(final_tf_idf[100000,:].toarray()[0])


[0. 0. 0. ... 0. 0. 0.]


Observation: as its a sparse matrix most of the vals are 0



Function to retrieve top 25 features/words for a given review


In [0]:
#Function to retrieve top 25 features/words for a given review
# source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row,features,top_n=25):
    
    #here argsort will return the top 25 features indices
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i],row[i]) for i in topn_ids]
    
    df = pd.DataFrame(top_feats)
    df.columns = ['features','tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[10,:].toarray()[0],features,25)


In [42]:
top_tfidf


,features,tfidf
0,with carol,0.194840
1,songs by,0.194840
2,quality kids,0.194840
3,kids storytelling,0.194840
4,heart quality,0.194840
5,or sound,0.194840
6,storytelling and,0.194840
7,king this,0.188815
8,these songs,0.188815
9,sound track,0.188815


In [43]:
# Using Google News Word2Vectors
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz', binary=True)


OSError: ignored

In [0]:
import pickle
pickle_out = open("word2vec_model","wb")
pickle.dump(model,pickle_out)
pickle_out.close()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
pickle_in=open("word2vec_model","rb")
model = pickle.load(pickle_in)

In [0]:
model.wv['computer']


NameError: ignored

In [0]:
model.wv.similarity('woman', 'man')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.76640123

In [0]:
model.wv.most_similar('woman')


NameError: ignored

In [0]:
model.wv.most_similar('tasti')

In [0]:
model.wv.most_similar('tasty')

In [0]:
# Train your own Word2Vec model using your own text corpus
import gensim
i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

In [0]:
# Train your own Word2Vec model using your own text corpus
import gensim
i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

In [45]:
print(final['Text'].values[0])
print("*****************************************************************")
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
*****************************************************************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is'

In [0]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)


In [47]:
words = list(w2v_model.wv.vocab)
print(len(words))


33783


In [48]:
w2v_model.wv.most_similar('tasty')


[('tastey', 0.9199376702308655),
 ('yummy', 0.858527421951294),
 ('satisfying', 0.8503388166427612),
 ('filling', 0.8282797336578369),
 ('delicious', 0.817266583442688),
 ('flavorful', 0.7996473908424377),
 ('tasteful', 0.7905883193016052),
 ('versatile', 0.7589945197105408),
 ('addicting', 0.7565521001815796),
 ('delectable', 0.7511836290359497)]

In [49]:
w2v_model.wv.most_similar('like')


[('resemble', 0.7238313555717468),
 ('dislike', 0.6384875774383545),
 ('mean', 0.6358470916748047),
 ('prefer', 0.6350363492965698),
 ('overpower', 0.6171197295188904),
 ('overwhelm', 0.6078854203224182),
 ('think', 0.604475736618042),
 ('alright', 0.6040796041488647),
 ('miss', 0.5975660681724548),
 ('enjoy', 0.5954923033714294)]

In [50]:
count_vect_feat = count_vect.get_feature_names() # list of words in the BoW
count_vect_feat.index('like')
print(count_vect_feat[64055])


activity great


In [51]:
w2v_model.wv.similarity('woman','man')


0.69589084

Explanation: 0-> both are different words 1-> exact similar words


In [52]:
# getting feature names from bag of words, count_vect has bow from the BOW code

count_vect_features = count_vect.get_feature_names() 
count_vect_features.index('like')


1442686

In [0]:
count_vect_features[64055]


'activity great'

Avg W2v, TFIDF-W2v

Avergae W2V is ntg but if given a sentence it calculates the w2v of every word in the sentence and sum it up then divide by no.of words which give me the avg W2V of the 'SENTENCE'.

Note: Avg W2V is used to get the Sentence Vector.

TFIDF weighted W2V we will calc w2v of a word in the sentece and multiply it with tfidf of that word, this is done for every word and is summed up and divided by sum of tfidf of every word.

Avg W2v = sum( w2v(wi) ) / (no.of words in the sentence)
Tf-IDF Weighted W2V = sum( ti * w2v(wi) ) / sum(ti)
Here 'ti' is tfidf of the word.



In [53]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of 50 length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)

print(len(sent_vectors)) #no.of reviews
print(len(sent_vectors[0])) #no.of dimensions

364171
50


In [0]:
import pickle


pickle_out = open("BOW_tfidf_avgW2V_TfidfW2V.pickle","wb")

pickle.dump(count_vect,pickle_out) #BOW
pickle.dump(final_counts,pickle_out) #BOW

pickle.dump(tf_idf_vect,pickle_out) #tfidf
pickle.dump(final_tf_idf,pickle_out) #tfidf
pickle.dump(features,pickle_out) #tfidf feature/ unique words


pickle.dump(w2v_model,pickle_out) #custom W2V model
pickle.dump(words,pickle_out) #custom W2V model

pickle.dump(sent_vectors,pickle_out) #avg W2V model


In [0]:
import pickle
pickleIn = open("listOfSentAfterCleaninhHTML_Punc.pickle","rb")
list_of_sent = pickle.load(pickleIn)

FileNotFoundError: ignored

In [0]:
pickle_in = open("BOW_tfidf_avgW2V_TfidfW2V.pickle","rb")
count_vect = pickle.load(pickle_in) #BOW
final_counts = pickle.load(pickle_in) #BOW

tf_idf_vect = pickle.load(pickle_in) #TFIDF
final_tf_idf = pickle.load(pickle_in) #TFIDF
features = pickle.load(pickle_in) #TFIDF

w2v_model = pickle.load(pickle_in) #w2v
words = pickle.load(pickle_in) #w2v

sent_vectors = pickle.load(pickle_in) #avg W2V

In [0]:
# TF-IDF weighted Word2Vec

tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            
            vec = w2v_model.wv[word]
            
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            
            sent_vec += (vec * tfidf)
            
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
   
    tfidf_sent_vectors.append(sent_vec)
    row += 1


In [0]:
pickle_out = open("WiightedTfidfW2V.pickle","wb")

pickle.dump(tfidf_sent_vectors,pickle_out) #tfidf weighted w2v model



pickle_out.close()


In [0]:
len(tfidf_sent_vectors)


In [0]:
tfidf_sent_vectors


In [0]:
# TF-IDF weighted Word2Vec
tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent[0:10000]: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            
            vec = w2v_model.wv[word]
            
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            
            sent_vec += (vec * tfidf)
            
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
   
    tfidf_sent_vectors.append(sent_vec)
    row += 1






In [0]:
pickle_out = open("WiightedTfidfW2V10kDatapoints.pickle","wb")

pickle.dump(tfidf_sent_vectors,pickle_out) #tfidf weighted w2v model

pickle_out.close()



In [0]:

len(tfidf_sent_vectors)

